In [1]:
import pandas as pd
import re
from datetime import timedelta
import sys
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import datetime

In [2]:
df=pd.read_excel("Lysol Radarly Export 1-7th June.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2757 entries, 0 to 2756
Columns: 149 entries, ID to TRANSLATED TEXT
dtypes: datetime64[ns](1), float64(90), int64(2), object(56)
memory usage: 3.1+ MB


In [3]:
df.head()

,ID,Date,Text,Title,Permalink,Site name,Queries,Language,Tone,Emotion(s),...,Content classification,Campaigns,IW - Global Topics,Adverse Effects,IW - Products topics,IW - BVC,IW - Topics split,Full Text,Language.1,TRANSLATED TEXT
0,1,2023-07-06 23:58:25,@MikeSington Gonna need hand sanitizer and Lys...,NaN,https://twitter.com/LynnRucker8/status/1666580...,http://twitter.com,Lysol,en,neutral,disgust,...,beauty_and_fitness;beauty_and_fitness.face_and...,NaN,NaN,NaN,NaN,NaN,NaN,@MikeSington Gonna need hand sanitizer and Lys...,en,@MikeSington Gonna need hand sanitizer and Lys...
1,2,2023-07-06 23:57:55,Got lysol?,NaN,https://nationalzero.com/2023/06/07/comer-punk...,http://nationalzero.com,Lysol,cy,neutral,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Got lysol?,en,Got lysol?
2,3,2023-07-06 23:57:21,"@Alphafox78 Moth balls, Lysol and mint julips.",NaN,https://twitter.com/VasWilder1/status/16665801...,http://twitter.com,Lysol,en,neutral,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"@Alphafox78 Moth balls, Lysol and mint julips.",en,"@Alphafox78 Moth balls, Lysol and mint julips."
3,4,2023-07-06 23:57:00,"In spite of the best cleansing, moist humid sp...","How indoor mold killer can Save You Time, Stre...",https://pet-friendly-mold-killer93603.onesmabl...,http://onesmablog.com,Lysol,en,neutral,NaN,...,home_and_garden.household_supplies;home_and_ga...,NaN,NaN,NaN,Bondage Products - Sex toys,CSR - Head (Summary);Sensory - Heart (Summary),Sexual Orientation - Trans,"In spite of the best cleansing, moist humid sp...",en,"In spite of the best cleansing, moist humid sp..."
4,5,2023-07-06 23:56:25,"@Alphafox78 cat piss, cigar smoke, anchovies a...",NaN,https://twitter.com/ApplesRBlue/status/1666579...,http://twitter.com,Lysol,en,negative,disgust,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"@Alphafox78 cat piss, cigar smoke, anchovies a...",en,"@Alphafox78 cat piss, cigar smoke, anchovies a..."


In [4]:
df.columns

Index(['ID', 'Date', 'Text', 'Title', 'Permalink', 'Site name', 'Queries',
       'Language', 'Tone', 'Emotion(s)',
       ...
       'Content classification', 'Campaigns', 'IW - Global Topics',
       'Adverse Effects', 'IW - Products topics', 'IW - BVC',
       'IW - Topics split', 'Full Text', 'Language.1', 'TRANSLATED TEXT'],
      dtype='object', length=149)

In [5]:
!pip install afinn

In [6]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [7]:
from textblob import TextBlob


In [8]:
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None

In [9]:
df['sentiment_polarity1'] = df['TRANSLATED TEXT'].apply(sentiment_calc)

In [10]:
sentiment_list = []
for i in df['sentiment_polarity1']:
    if i > 0:
        sentiment_list.append('Positive')
    elif i < 0:
        sentiment_list.append('Negative')
    else:
        sentiment_list.append('Neutral')
df['sentiment_t']=sentiment_list
        

In [11]:
pd.crosstab(index=df["sentiment_t"],columns="count") 

col_0,count
sentiment_t,
Negative,426
Neutral,592
Positive,1739


In [12]:
from afinn import Afinn
af = Afinn()


In [13]:
def sentiment_calc1(text):
    try:
        return af.score(text)
    except:
        return None
    
df['sentiment_a']=sentiment_list    

In [14]:
df['sentiment_polarity1'] = df['TRANSLATED TEXT'].apply(sentiment_calc1)

In [15]:
pd.crosstab(index=df["sentiment_a"],columns="count") 

col_0,count
sentiment_a,
Negative,426
Neutral,592
Positive,1739


In [16]:
!pip install vaderSentiment

In [17]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [18]:
sid_obj = SentimentIntensityAnalyzer()

In [19]:
def sentiment_scores(sentence):
    try:
        return sid_obj.polarity_scores(sentence) 
    except:
        return None

In [20]:
sentiment_dict = df['TRANSLATED TEXT'].apply(sentiment_scores)

In [21]:
df["senti_dict"]=sentiment_dict

In [22]:
#sentiment_dict[0]["compound"]

In [23]:
a=[]
for i in range(len(df["senti_dict"])):
    try:
        a.append(df["senti_dict"][i]['compound'])
    except:
        a.append("NA")


In [24]:
df["van_score"]=a

In [25]:
df.head(20)

,ID,Date,Text,Title,Permalink,Site name,Queries,Language,Tone,Emotion(s),...,IW - BVC,IW - Topics split,Full Text,Language.1,TRANSLATED TEXT,sentiment_polarity1,sentiment_t,sentiment_a,senti_dict,van_score
0,1,2023-07-06 23:58:25,@MikeSington Gonna need hand sanitizer and Lys...,NaN,https://twitter.com/LynnRucker8/status/1666580...,http://twitter.com,Lysol,en,neutral,disgust,...,NaN,NaN,@MikeSington Gonna need hand sanitizer and Lys...,en,@MikeSington Gonna need hand sanitizer and Lys...,0.0,Neutral,Neutral,"{'neg': 0.0, 'neu': 0.758, 'pos': 0.242, 'comp...",0.4939
1,2,2023-07-06 23:57:55,Got lysol?,NaN,https://nationalzero.com/2023/06/07/comer-punk...,http://nationalzero.com,Lysol,cy,neutral,NaN,...,NaN,NaN,Got lysol?,en,Got lysol?,0.0,Neutral,Neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
2,3,2023-07-06 23:57:21,"@Alphafox78 Moth balls, Lysol and mint julips.",NaN,https://twitter.com/VasWilder1/status/16665801...,http://twitter.com,Lysol,en,neutral,NaN,...,NaN,NaN,"@Alphafox78 Moth balls, Lysol and mint julips.",en,"@Alphafox78 Moth balls, Lysol and mint julips.",0.0,Neutral,Neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
3,4,2023-07-06 23:57:00,"In spite of the best cleansing, moist humid sp...","How indoor mold killer can Save You Time, Stre...",https://pet-friendly-mold-killer93603.onesmabl...,http://onesmablog.com,Lysol,en,neutral,NaN,...,CSR - Head (Summary);Sensory - Heart (Summary),Sexual Orientation - Trans,"In spite of the best cleansing, moist humid sp...",en,"In spite of the best cleansing, moist humid sp...",30.0,Positive,Positive,"{'neg': 0.034, 'neu': 0.824, 'pos': 0.142, 'co...",0.996
4,5,2023-07-06 23:56:25,"@Alphafox78 cat piss, cigar smoke, anchovies a...",NaN,https://twitter.com/ApplesRBlue/status/1666579...,http://twitter.com,Lysol,en,negative,disgust,...,NaN,NaN,"@Alphafox78 cat piss, cigar smoke, anchovies a...",en,"@Alphafox78 cat piss, cigar smoke, anchovies a...",-4.0,Neutral,Neutral,"{'neg': 0.278, 'neu': 0.722, 'pos': 0.0, 'comp...",-0.4019
5,6,2023-07-06 23:50:51,"@Alphafox78 An invigorating mix of Lysol, carp...",NaN,https://twitter.com/MPWG91655219/status/166657...,http://twitter.com,Lysol,en,positive,disgust,...,NaN,NaN,"@Alphafox78 An invigorating mix of Lysol, carp...",en,"@Alphafox78 An invigorating mix of Lysol, carp...",1.0,Positive,Positive,"{'neg': 0.0, 'neu': 0.69, 'pos': 0.31, 'compou...",0.6597
6,7,2023-07-06 23:48:14,How to ditch the chemical cleaning products in...,8 Ways To Ditch The Chemical Cleaning Products,https://notoxminimalist.com/ditch-chemical-cle...,http://notoxminimalist.com,Lysol,en,neutral,NaN,...,Product - Head (Summary);Sensory - Heart (Summ...,Sexual Orientation - Trans,How to ditch the chemical cleaning products in...,en,How to ditch the chemical cleaning products in...,14.0,Positive,Positive,"{'neg': 0.088, 'neu': 0.801, 'pos': 0.111, 'co...",0.9845
7,8,2023-07-06 23:44:59,@Civil_Roar_ It looks alot like my aunts livin...,NaN,https://twitter.com/EvilJohnLives/status/16665...,http://twitter.com,Lysol,en,negative,disgust,...,Sensory - Heart (Summary),NaN,@Civil_Roar_ It looks alot like my aunts livin...,en,@Civil_Roar_ It looks alot like my aunts livin...,4.0,Neutral,Neutral,"{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'comp...",0.6124
8,9,2023-07-06 23:44:56,"During the summer of 2020, still in lockdown, ...",NaN,https://www.reddit.com/r/mildlyinfuriating/com...,http://www.reddit.com,Lysol,en,negative,NaN,...,NaN,NaN,"During the summer of 2020, still in lockdown, ...",en,"During the summer of 2020, still in lockdown, ...",8.0,Positive,Positive,"{'neg': 0.022, 'neu': 0.857, 'pos': 0.121, 'co...",0.9216
9,10,2023-07-06 23:44:47,mousedetective:\nPlease Help A Mostly Queer Ho...,NaN,https://onedumbazz.tumblr.com/post/71950966460...,http://onedumbazz.tumblr.com,Lysol,en,neutral,NaN,...,Sensory - Heart (Summary),Sexual Orientation - Queer,mousedetective:\nPlease Help A Mostly Queer Ho...,en,mousedetective:\nPlease Help A Mostly Queer Ho...,1

In [26]:
s_list=[]
for i in df["van_score"]:
    try:
        if i > 0.1:
            s_list.append('Positive')
        elif i < 0:
            s_list.append('Negative')
        else:
            s_list.append('Neutral')
    except:
            s_list.append('NA')

        

In [27]:
df['sentiment1']=s_list

In [28]:
pd.crosstab(index=df["sentiment1"],columns="count") 


col_0,count
sentiment1,
NA,33
Negative,527
Neutral,487
Positive,1710


In [29]:
df['Analytics Sentiment']=df[['sentiment1','sentiment_t']].mode(axis=1)[0]

In [30]:
pd.crosstab(index=df['Analytics Sentiment'],columns="count") 

col_0,count
Analytics Sentiment,
NA,33
Negative,702
Neutral,555
Positive,1467


In [31]:
df1=df.drop(['sentiment_polarity1','sentiment_a','sentiment_t',
        'senti_dict', 'van_score',
       'sentiment1'], axis = 1) 

In [32]:
writer = pd.ExcelWriter(r"Sentiment_Lysol Radarly Export 1-7th June.xlsx", engine='xlsxwriter',options={'strings_to_urls': False})
df1.to_excel(writer)
writer.close()

C:\Users\ujjwal.pratik1\AppData\Local\Temp\ipykernel_14028\929402712.py:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter(r"Sentiment_Lysol Radarly Export 1-7th June.xlsx", engine='xlsxwriter',options={'strings_to_urls': False})


In [33]:
df1.isnull().sum()

ID                        0
Date                      0
Text                     10
Title                  1699
Permalink                 6
                       ... 
IW - Topics split      2270
Full Text                10
Language.1                0
TRANSLATED TEXT          33
Analytics Sentiment       0
Length: 150, dtype: int64

In [34]:
df1

,ID,Date,Text,Title,Permalink,Site name,Queries,Language,Tone,Emotion(s),...,Campaigns,IW - Global Topics,Adverse Effects,IW - Products topics,IW - BVC,IW - Topics split,Full Text,Language.1,TRANSLATED TEXT,Analytics Sentiment
0,1,2023-07-06 23:58:25,@MikeSington Gonna need hand sanitizer and Lys...,NaN,https://twitter.com/LynnRucker8/status/1666580...,http://twitter.com,Lysol,en,neutral,disgust,...,NaN,NaN,NaN,NaN,NaN,NaN,@MikeSington Gonna need hand sanitizer and Lys...,en,@MikeSington Gonna need hand sanitizer and Lys...,Neutral
1,2,2023-07-06 23:57:55,Got lysol?,NaN,https://nationalzero.com/2023/06/07/comer-punk...,http://nationalzero.com,Lysol,cy,neutral,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Got lysol?,en,Got lysol?,Neutral
2,3,2023-07-06 23:57:21,"@Alphafox78 Moth balls, Lysol and mint julips.",NaN,https://twitter.com/VasWilder1/status/16665801...,http://twitter.com,Lysol,en,neutral,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"@Alphafox78 Moth balls, Lysol and mint julips.",en,"@Alphafox78 Moth balls, Lysol and mint julips.",Neutral
3,4,2023-07-06 23:57:00,"In spite of the best cleansing, moist humid sp...","How indoor mold killer can Save You Time, Stre...",https://pet-friendly-mold-killer93603.onesmabl...,http://onesmablog.com,Lysol,en,neutral,NaN,...,NaN,NaN,NaN,Bondage Products - Sex toys,CSR - Head (Summary);Sensory - Heart (Summary),Sexual Orientation - Trans,"In spite of the best cleansing, moist humid sp...",en,"In spite of the best cleansing, moist humid sp...",Positive
4,5,2023-07-06 23:56:25,"@Alphafox78 cat piss, cigar smoke, anchovies a...",NaN,https://twitter.com/ApplesRBlue/status/1666579...,http://twitter.com,Lysol,en,negative,disgust,...,NaN,NaN,NaN,NaN,NaN,NaN,"@Alphafox78 cat piss, cigar smoke, anchovies a...",en,"@Alphafox78 cat piss, cigar smoke, anchovies a...",Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,2753,2023-01-06 00:07:22,What have me screaming is the fuckin Lysol goi...,NaN,https://twitter.com/thakingtee/status/16640459...,http://twitter.com,Lysol,en,negative,anger;disgust,...,NaN,NaN,NaN,NaN,NaN,NaN,What have me screaming is the fuckin Lysol goi...,en,What have me screaming is the fuckin Lysol goi...,Negative
2753,2754,2023-01-06 00:04:48,I use Lysol Laundry Sanitizer when anything fa...,NaN,https://www.reddit.com/r/dyeing/comments/13w02...,http://www.reddit.com,Lysol,en,negative,disgust,...,NaN,NaN,NaN,NaN,Sensory - Heart (Summary),NaN,I use Lysol Laundry Sanitizer when anything fa...,en,I use Lysol Laundry Sanitizer when anything fa...,Negative
2754,2755,2023-01-06 00:03:04,@wesruss .....and that Lysol commercial came o...,NaN,https://twitter.com/OldRadioDude2/status/16640...,http://twitter.com,Lysol,en,negative,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,@wesruss .....and that Lysol commercial came o...,en,@wesruss .....and that Lysol commercial came o...,Positive
2755,2756,2023-01-06 00:01:41,Keep Lysol in car. Streptococcus Throat is goi...,NaN,/1.0/facebook-comment-proxy?pageId=24483084229...,http://facebook.com,Lysol,en,neutral,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Keep Lysol in car. Streptococcus Throat is goi...,en,Keep Lysol in car. Streptococcus Throat is goi...,Positive
